# Singleton & TPL

- Task Parallel Library; https://learn.microsoft.com/en-us/dotnet/standard/parallel-programming/task-parallel-library-tpl
- Task based async programming; https://learn.microsoft.com/en-us/dotnet/standard/parallel-programming/task-based-asynchronous-programming

In [ ]:
#r "nuget: Microsoft.EntityFrameworkCore.SqlServer, *-*"

In [ ]:
using Microsoft.EntityFrameworkCore;

class Reading
{
    public int DataId { get; set; }

    public int Number1 { get; set; }
    public int Number2 { get; set; }
    public int Number3 { get; set; }
    public int Number4 { get; set; }
    public int Number5 { get; set; }

    public double WideNumber1 { get; set; }
    public double WideNumber2 { get; set; }
    public double WideNumber3 { get; set; }
    public double WideNumber4 { get; set; }
    public double WideNumber5 { get; set; }

    public string String1 { get; set; }
    public string String2 { get; set; }
    public string String3 { get; set; }
    public string String4 { get; set; }
    public string String5 { get; set; }
}

class JiraDbContext : DbContext
{
    public DbSet<Reading> Readings { get; set; }

    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
    {
        var connectionString = "Server=.;Database=JiraDb;Trusted_Connection=True;TrustServerCertificate=True;";
        optionsBuilder.UseSqlServer(connectionString);
    }

    protected override void OnModelCreating(ModelBuilder modelBuilder)
    {
        //base.OnModelCreating(modelBuilder);
        modelBuilder.Entity<Reading>(entity =>
        {
            entity.HasKey(e => e.DataId);
            entity.HasIndex(e => e.DataId).IsUnique();
        });
    }
}

In [ ]:
var random = new Random(Guid.NewGuid().GetHashCode());

using(var db = new JiraDbContext())
{
    db.Database.EnsureCreated();

    for(int i = 1; i <= 25000; i ++) //lets generate lot of data
    {
        db.Readings.Add(new Reading()
        {
            Number1 = random.Next(), Number2 = random.Next(), Number3 = random.Next(),
            Number4 = random.Next(), Number5 = random.Next(),

            WideNumber1 = random.Next() * random.Next(),
            WideNumber2 = random.Next() * random.Next(),
            WideNumber3 = random.Next() * random.Next(),
            WideNumber4 = random.Next() * random.Next(),
            WideNumber5 = random.Next() * random.Next(),

            String1 = random.Next().GetHashCode().ToString(),
            String2 = random.Next().GetHashCode().ToString(),
            String3 = random.Next().GetHashCode().ToString(),
            String4 = random.Next().GetHashCode().ToString(),
            String5 = random.Next().GetHashCode().ToString()
        });
        db.SaveChanges();
    }
}

In [ ]:
using System.Threading;

class Singleton
{
    static readonly object lockObject = new object();
    static volatile Singleton instance; //volatile importance

    Reading[] data = null;

    void print(string s)
    {
        Console.WriteLine($"{DateTime.Now} {s}");
    }

    Singleton() 
    {
        print("ctor started");
        using(var db = new JiraDbContext())
        {
            var q = from d in db.Readings
                    select d;
            data = q.ToArray();
        }
        print("ctor finished");
    }

    public Reading[] Data => data;

    public static Singleton Instance
    {
        get
        {
            if (instance == null)
            {
                lock (lockObject)
                {
                    if (instance == null)
                    {
                        instance = new Singleton();
                    }
                }
            }
            return instance;
        }
    }
}

In [ ]:
using System.Linq;

void firstRandomRow()
{
    var randomRow = Singleton.Instance.Data.OrderBy(d => Guid.NewGuid()).FirstOrDefault();
    if (null != randomRow)
        Console.WriteLine($"{DateTime.Now} [{Thread.CurrentThread.ManagedThreadId}] {randomRow.DataId} is selected");
}

var tasks = new []
{
    new Task(firstRandomRow),
    new Task(firstRandomRow),
    new Task(firstRandomRow),
    new Task(firstRandomRow),
    new Task(firstRandomRow)
};

Parallel.ForEach(tasks, task =>
{
    task.Start();
});

await Task.WhenAll(tasks).ContinueWith(done =>
{
    Console.WriteLine($"{DateTime.Now} Completed...");
});